In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from torch.utils.data import Dataset
import torch
import os
import random
import numpy as np
from torch import nn
from typing import Dict, Optional, Tuple, List
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, random_split
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import precision_score, f1_score, recall_score, classification_report

import time
import math
import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import gc

from src.data_loader import *
from src.qa_dataset import *
from src.train import *
from src.classifiers import *
from src.graph import *

/home/razvor/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 42

torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
gc.collect()
torch.cuda.empty_cache()

## Load data

In [5]:
train_dev_path = "../data/tsv/train.tsv"
test_path = "../data/tsv/test.tsv"

train_df, dev_df, test_df = load_and_split(train_dev_path, test_path)

Questions: train - 3181, dev - 354, test - 1000
Train: (33911, 10)
Dev: (3761, 10)
Test: (10961, 7)


/mnt/c/Users/razvor/Documents/DS/Skoltech/NLP course/TextGraphs17-shared-task-main/my_experiments/src/data_loader.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["label"] = train_df["correct"].astype(np.float32)
/mnt/c/Users/razvor/Documents/DS/Skoltech/NLP course/TextGraphs17-shared-task-main/my_experiments/src/data_loader.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev_df["label"] = dev_df["correct"].astype(np.float32)
/mnt/c/Users/razvor/Documents/DS/Skoltech/NLP course/TextGraphs1

In [6]:
train_df.loc[2, "graph"]['links']

[{'name_': 'P17', 'source': 0, 'target': 0, 'label': 'country'},
 {'name_': 'P35', 'source': 0, 'target': 1, 'label': 'head of state'},
 {'name_': 'P27', 'source': 1, 'target': 0, 'label': 'country of citizenship'}]

In [7]:
#sep = tokenizer.sep_token
sep = '|'
linearize_graph = linearize_graph_gen(sep)
train_df["linearized_graph"] = train_df["graph"].apply(linearize_graph)
dev_df["linearized_graph"] = dev_df["graph"].apply(linearize_graph)
test_df["linearized_graph"] = test_df["graph"].apply(linearize_graph)

In [8]:
train_df["num_internal"] = train_df["graph"].apply(get_graph_num_internal)
dev_df["num_internal"] = dev_df["graph"].apply(get_graph_num_internal)
test_df["num_internal"] = test_df["graph"].apply(get_graph_num_internal)

train_df["num_links"] = train_df["graph"].apply(get_num_links)
dev_df["num_links"] = dev_df["graph"].apply(get_num_links)
test_df["num_links"] = test_df["graph"].apply(get_num_links)

In [9]:
#train_df.to_csv('./data/train.csv', index=False)
#dev_df.to_csv('./data/dev.csv', index=False)
#test_df.to_csv('./data/test.csv', index=False)

## Init model

In [10]:
model_name="sentence-transformers/all-mpnet-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)

freeze_embeddings = True
if freeze_embeddings:
    for param in bert_model.embeddings.parameters():
        param.requires_grad = False

freeze_layer_count = 10
if freeze_layer_count > 0:
    for layer in bert_model.encoder.layer[:freeze_layer_count]:
        for param in layer.parameters():
            param.requires_grad = False

print("# Trainable params: ", sum(p.numel() for p in bert_model.parameters() if p.requires_grad))

# Trainable params:  14766720


In [21]:
max_length = 256
train_dataset = QuestionAnswerDataset(train_df, tokenizer=tokenizer, max_length=max_length, context_key="linearized_graph",
                                      tokenizer_truncation="only_second", add_meta = None)
dev_dataset = QuestionAnswerDataset(dev_df, tokenizer=tokenizer, max_length=max_length, context_key="linearized_graph",
                                    tokenizer_truncation="only_second", add_meta = None)
test_dataset = QuestionAnswerDataset(test_df, tokenizer=tokenizer, max_length=max_length, context_key="linearized_graph",
                                     tokenizer_truncation="only_second", add_meta = None)

In [12]:
idx_bound = 2
for i in range(idx_bound):
    d = train_dataset[i]
    inp_ids = d["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(inp_ids)
    s = "".join((x.strip("#") if x.startswith("#") else f" {x}" for x in tokens))
    print(s)
    d = dev_dataset[i]
    inp_ids = d["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(inp_ids)
    s = "".join((x.strip("#") if x.startswith("#") else f" {x}" for x in tokens))
    print(s)
    d = test_dataset[i]
    inp_ids = d["input_ids"]
    tokens = tokenizer.convert_ids_to_tokens(inp_ids)
    s = "".join((x.strip("#") if x.startswith("#") else f" {x}" for x in tokens))
    print(s)
    print('---')

 <s> whst is the name of the head of state and highest ranking political and religious authority in iran ? </s> </s> iran , country , iran iran , replaces , pahlavi dynasty pahlavi dynasty , replaced by , iran | ruhollah khomeini ' s return to iran | , country , pahlavi dynasty </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

In [12]:
"""
(*) """.join(train_df.head(10).linearized_graph.tolist())

" Iran, country, Iran  Iran, replaces, Pahlavi dynasty  Pahlavi dynasty, replaced by, Iran  </s> Ruhollah Khomeini's return to Iran </s>, country, Pahlavi dynasty \n(*)  Iran, described by source, Jewish Encyclopedia of Brockhaus and Efron  Iran, country, Iran  letter, described by source, Jewish Encyclopedia of Brockhaus and Efron  letter, subclass of, written work  The Book of Healing, country of origin, Iran  The Book of Healing, instance of, written work  </s> Ruhollah Khomeini's letter to Mikhail Gorbachev </s>, instance of, written work  </s> Ruhollah Khomeini's letter to Mikhail Gorbachev </s>, instance of, letter \n(*)  Iran, country, Iran  Iran, head of state, </s> Ruhollah Khomeini </s>  </s> Ruhollah Khomeini </s>, country of citizenship, Iran \n(*)  Iran, country, Iran  </s> Office of the Supreme Leader of Iran </s>, different from, Imam Khomeini Hussainiya  Imam Khomeini Hussainiya, country, Iran  Imam Khomeini Hussainiya, part of, </s> Office of the Supreme Leader of Iran

In [22]:
batch_size = 64
num_workers = 0

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True,
)
dev_loader = torch.utils.data.DataLoader(
    dev_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False,
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False,
)

In [23]:
tokenizer.sep_token

'</s>'

## Train

In [24]:
N_EPOCHS = 3
DROPOUT = 0.1

In [30]:
bert_linearized_graph_clf = BertSimpleClassifier(bert_model, dropout=DROPOUT, add_meta = None).to(device)
optimizer = optim.Adam(bert_linearized_graph_clf.parameters(), lr=3e-5)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
train_evaluate(bert_linearized_graph_clf, train_loader, dev_loader, optimizer,
               criterion, N_EPOCHS, "bert_linearized_graph_clf", device)

## Evaluate on dev

In [35]:
true_labels, pred_labels = predict(bert_linearized_graph_clf, dev_loader, device)
print(f"{precision_score(true_labels, pred_labels)},{recall_score(true_labels, pred_labels)},{f1_score(true_labels, pred_labels)}")

0.6713881019830028,0.6220472440944882,0.6457765667574932


In [3]:
0.66/0.645*100

102.32558139534885